In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update > /dev/null
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-30 19:17:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-30 19:17:02 (6.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [25]:
jdbc_url="jdbc:postgresql://----/my_data_class_db"
df = spark.read.format("jdbc").option("url",jdbc_url).option("dbtable", "vine_table").option("user","----").option("password","----").load()
print(df.count())
df.show()

1785997
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|  

In [26]:
df_filter = df[df.total_votes > 20]
print(df_filter.count())
df_filter.show()

61153
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N

In [12]:
div_df = df_filter[(df_filter.helpful_votes/df_filter.total_votes) >= 0.5]
print(div_df.count())
div_df.show()


37921
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N

In [13]:
vine_program_df = div_df[div_df.verified_purchase == 'Y']
print(vine_program_df.count())
vine_program_df.show()

9454
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|                Y|
|R16621F16PQN5A|          3|           24|         31|   N|                Y|
| RNEMGSUKJ4535|          2|           21|         25|   N|                Y|
|R3R2UKBFKP1IQ4|          1|           19|         23|   N|                Y|
|R2T90XM5G2DBZA|          3|           15|         22|   N|                Y|
|R12E2W0XOV3I2S|          1|           24|         30|   N|

In [15]:
no_vine_program_df = div_df[div_df.verified_purchase == 'N']
print(no_vine_program_df.count())
no_vine_program_df.show()

28467
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R2OWDAR0EDEPTF|          1|           23|         28|   N|                N|
|R29BOS5HMAY1LO|          5|           88|        110|   N|                N|
| R8WD0136CETA7|          2|           99|        119|   N|                N|
|R1LXBPG1M9DDOC|          1|          102|        106|   N|                N|
| R5ZMK0Y0KRBRZ|          1|           79|         91|   N

In [19]:
five_star_paid = vine_program_df[vine_program_df.star_rating == 5].count()
no_five_star_paid = vine_program_df[vine_program_df.star_rating != 5].count()
percentage = (five_star_paid / no_five_star_paid) * 100
print('Number of 5 stars: ' + str(five_star_paid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 3607 Percentage: 61.69


In [23]:
five_star_unpaid = no_vine_program_df[no_vine_program_df.star_rating == 5].count()
no_five_star_unpaid = no_vine_program_df[no_vine_program_df.star_rating != 5].count()
percentage = (five_star_unpaid / no_five_star_unpaid) * 100
print('Number of 5 stars: ' + str(five_star_unpaid), 'Percentage: ' + str(round(percentage,2)))


Number of 5 stars: 11141 Percentage: 64.3
